In [1]:
import os
import numpy as np
import sklearn
import pandas as pd
import pickle

In [2]:
# #Initializing file paths
raw_dir = os.path.join(os.pardir,os.pardir,'data','raw')
processed_dir = os.path.join(os.pardir,os.pardir,'data','processed')
model_dir = os.path.join(os.pardir,os.pardir,'models')

In [4]:
# train_path = "https://worksheets.codalab.org/rest/bundles/0x7b873062afd04a7f82a7a49940ee7737/contents/blob/" #'../../data/raw/train.csv'
# val_path = "https://worksheets.codalab.org/rest/bundles/0xa63401efaa6d44e39ed6ed9fe7e08cd2/contents/blob/"#'../../data/raw/dev.csv'
# test_path = "https://worksheets.codalab.org/rest/bundles/0xbb4a5897ea4b456fb7a6bd81289d1fbc/contents/blob/"

train_path = os.path.join(raw_dir,'train.csv')
val_path = os.path.join(raw_dir, 'dev.csv')

train = pd.read_csv(train_path)
validation = pd.read_csv(val_path)
# test = pd.read_csv(test_path)

In [5]:
train.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...


In [6]:
validation.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,11,934,0,5.0,1,2014-01-20,"all around good place, cozy, I came in and did..."
1,17,940,0,4.0,0,2014-09-16,"For lunch, my friend and I had: -Lamb sandwich..."
2,20,943,0,5.0,0,2014-05-24,Some good Big Greek cooking!! Came to City on ...
3,30,953,0,4.0,0,2013-10-17,So... as you may notice from some of my other ...
4,43,966,0,3.0,0,2012-12-19,"I don't understand the whole ""You can't order ..."


In [7]:
seed = 10

In [8]:
positive_examples = train[train['label']==1]

# Setting frac = 1 to shuffle all the data
negative_examples = train[train['label']==0]

In [10]:
# Sampling thesame size of date in positive class from the negative class
sampled_negative_examples = negative_examples.sample(n=positive_examples.shape[0],random_state=seed)

In [11]:
# balanced data
balanced_df = pd.concat([sampled_negative_examples, positive_examples])
# balanced_df.head()

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
# Balanced data
bal_cnt_vectorizer = CountVectorizer(stop_words='english', binary=True)
bal_tfidf_vectorizer = TfidfVectorizer(stop_words='english', binary=True)
bal_cnt_vectorizer.fit(balanced_df['review'])
bal_tfidf_vectorizer.fit(balanced_df['review'])

bal_cnt_X_train = bal_cnt_vectorizer.transform(balanced_df['review'])
bal_tfidf_X_train = bal_tfidf_vectorizer.transform(balanced_df['review'])

bal_cnt_X_dev = bal_cnt_vectorizer.transform(validation['review'])
bal_tfidf_X_dev = bal_tfidf_vectorizer.transform(validation['review'])

bal_Y_train = balanced_df['label']
bal_Y_dev = validation['label']

In [13]:
from sklearn.naive_bayes import MultinomialNB 

In [14]:
tfidf_nb = MultinomialNB()
cnt_nb = MultinomialNB()

In [23]:
tfidf_nb = tfidf_nb.fit(bal_tfidf_X_train,bal_Y_train)
cnt_nb = cnt_nb.fit(bal_cnt_X_train, bal_Y_train)

In [24]:
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

In [25]:
def ClassifierMetrics (X_train, Y_train, X_test, Y_test, fitted_model):
    """
    X_train: training set features
    Y_train: training set labels
    X_test: dev/test set features
    Y_test: dev/test set labels
    fitted_model: 
    """
    Y_pred = fitted_model.predict(X_test.toarray())
    Y_score = fitted_model.predict_proba(X_test.toarray())[:,1]
    metrics = {'train_accuracy': fitted_model.score(X_train.toarray(), Y_train),
               'test_accuracy': fitted_model.score(X_test.toarray(), Y_test),
               'test_auc_pred': roc_auc_score(Y_test, Y_pred),
               'test_auc_score': roc_auc_score(Y_test, Y_score),
               'test_ap_pred': average_precision_score(Y_test, Y_pred),
               'test_ap_score': average_precision_score(Y_test, Y_score)}
    return metrics, Y_pred

In [26]:
bal_tfidf_metrics, bal_tfidf_pred = ClassifierMetrics (bal_tfidf_X_train, bal_Y_train, bal_tfidf_X_dev, bal_Y_dev, tfidf_nb)
bal_cnt_metrics, bal_cnt_pred = ClassifierMetrics (bal_cnt_X_train, bal_Y_train, bal_cnt_X_dev, bal_Y_dev, cnt_nb)


MemoryError: 